In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("data/amazon_test.csv").fillna("")
_, sample = train_test_split(df, test_size=0.1, stratify=df.label)
#df_train_sample.to_csv("data/yelp_train_sample.csv", index=False)

In [4]:
sample.to_csv("data/amazon_test_sample.csv", index=False)
#sample.label.value_counts()